<a href="https://colab.research.google.com/github/WellML/detectioncable/blob/master/20200427_YOLOv3_Custom_Cable_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stand: 22.04.2020

Autor: Thomas Bednarski
# Projekt: **YOLOv3 Custom Cable Detection**

Nutzung von Google Colaboratory (Colab) zur Anwendung von YOLOv3 Objekterkennung für Kabelenden:

>Crimp, 

>Crimp_Seal, 

>Isolation, 

>Stripped_Isolation.



Inhaltsverzeichnis:
1. Erste Schritte
2. Voreinstellungen (jedes Mal neu durchzuführen)
3. YOLOv3 Custom Cable Detection (Masterarbeit)
4. Eigenes Modell trainieren

# **1) Voreinstellungen Colab (einmalig)**
### Projekt in eigenes Google-Konto transferieren

> Voraussetzung: Eigenes Google Konto

Zur Nutzung und weiteren Bearbeitung von YOLOv3 Custom Cable Detection muss dieses Notebook in das eigene Google-Konto zu transferiert werden, um Google Drive (GDrive) als Speicherort zu nutzen. GDrive wird darauffolgendend als Speicherort von Bildern und Trainingsmaterial verwendet.
> 1. Klicke auf "Open in playground"
> 2. Klicke auf "Copy to Drive"


### GPU in Colab aktivieren
Der Hauptgrund zur Anwendung von YOLOv3 in Colab ist die Nutzung der GPU, die Google innerhalb eines Notebook-Projekts zur Verfügung stellt. Dieser Schritt muss nur einmalig getätigt werden.

>Die GPU lässt sich aktivieren in: **Edit > Notebook Settings**

>1. Runtime type: *Python 3*
>2. Hardware accelerator: *GPU*


Prüfe, ob die GPU von Colab genutzt wird. [Quelle (1. Setup)](https://github.com/robingenz/object-detection-yolov3-google-colab/blob/master/Object_Detection_YOLOv3_Google_Colab.ipynb)

In [0]:
#import tensorflow as tf
#device_name = tf.test.gpu_device_name()
#if not '/device:GPU:0' in device_name:
#    print('\nFEHLER: Bitte wähle die GPU in Notebook Settings und starte erneut!')
#    exit
#else:
#    !nvidia-smi
#    print('OK')

# **2) Voreinstellungen Darknet (jedes Mal nach Neustart des Projekts durchzuführen)**

### Inaktivität in Colab überbrücken

Damit Colab nicht automatisch die Verbindung trennt, kann dem Browser durch Code suggeriert werden, dass jede 10 Sekunden ein Mausklick durchgeführt wird. [Quelle](https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0)

>Drücke **STRG + SHIFT + i**.

>Gebe folgenden Code im Tab "**Console**" ein.

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```

>Klicke **Enter**. (Das Fenster offen lassen)

Eine beliebige Zahl erscheint als Output. Darunter folgt nach einiger Zeit **Working** und einer Zahl davor, die von 1 aufwärts zählt. Damit wird verhindert, dass Colab die Verbindung aufgrund von Inaktivität trennt.


###Automatisierung
> Klicke **STRG + F9**, um alle Codes in *2) Voreinstellungen* und ggf. *4) Training* nacheinander auszuführen.

> Klicke auf erscheinenden Link unter "GDrive mit Colab verknüpfen", kopiere den Bestätigungscode in das Eingabefeld und bestätige mit Enter.

Die folgenden Codes werden automatisch mithilfe **STRG + F9** durchgeführt und es ist keine eigene Aktion mehr nötig.

## GDrive mit Colab verknüpfen
Als Speicherort von Bildern und Trainingsmaterial eignet sich GDrive sehr gut, da Colab dort auf diese Bilder zugreifen sowie erstellte Bilderkennungen ablegen kann. Jedoch muss Colab zunächst berechtigt werden auf GDrive zugreifen zu dürfen.




In [0]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Nachdem Colab Zugriff auf GDrive besitzt, werden Ordner erstellt, falls nicht bereits vorhanden.

In [0]:
%cd /content/gdrive/My\ Drive
%mkdir -p /content/gdrive/My\ Drive/yolocable/backup
%mkdir -p /content/gdrive/My\ Drive/yolocable/detected_images
%mkdir -p /content/gdrive/My\ Drive/yolocable/original_images
print('OK')

/content/gdrive/My Drive
OK


In [0]:
!ln -s /content/gdrive/My\ Drive/yolocable /gdriveyolo
!ls /gdriveyolo

01_yolov3_custom_best.weights  backup		  original_images
02_yolov3_custom_best.weights  cable.data	  test.txt
20200408_cable.weights	       cable.names	  train_dataset.rar
20200420_cable.weights	       darknet		  train.txt
20200421_cable.weights	       darknet53.conv.74  yolov3_custom.cfg
20200422_cable.weights	       detected_images	  yolov3_custom_last.weights


### Konfiguration und Installation von Darknet

Die folgenden Codes installieren YOLOv3 innerhalb dieses Notebook-Projekts.
Zunächst wird Darknet heruntergeladen und danach im Makefile OPENCV und GPU für YOLOv3 aktiviert, indem die Zahl 0 durch 1 ersetzt wird. Diese Vorgehensweise ist identisch zur Eingabe über die Konsole beim Betriebssystem Linux auf dem Rechner. Im Gegensatz zum privaten Rechner, bei dem lediglich die CPU-Nutzung möglich war, ist nun die schnellere GPU-Leistung in Colab möglich. "!" und "%" markieren jeweils den Anfang der Codezeile äquivalent zur Linux-Konsole "$".

In [0]:
%cd /content
#!git clone https://github.com/pjreddie/darknet
#!git clone https://github.com/AlexeyAB/darknet
!git clone https://github.com/WellML/darknet
%cd darknet

/content
Cloning into 'darknet'...
remote: Enumerating objects: 13057, done.
remote: Total 13057 (delta 0), reused 0 (delta 0), pack-reused 13057
Receiving objects: 100% (13057/13057), 11.76 MiB | 11.56 MiB/s, done.
Resolving deltas: 100% (8922/8922), done.
/content/darknet


In [0]:
!apt-get update
!apt-get upgrade
!apt-get install build-essential
!apt-get install cmake git libgtk2.0-dev pkg-config libavcodec-dev libavformat-dev libswscale-dev
!apt-get install libavcodec-dev libavformat-dev libswscale-d
!apt-get install libopencv-dev

%mkdir custom
%mkdir cable
!ln -s /content/darknet /dn

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
print('OK')

%cd ../
!ls
!apt install g++-5
!apt install gcc-5
!apt update
!apt upgrade


Get:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [37.4 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubu

Bevor die Anwendung begonnen werden kann, müssen vorher die relevanten Dateien aus Github nach Darknet mit folgendem Code kopiert werden.

In [0]:
%cd /content/
!cp -r darknet /gdriveyolo/

/content


In [0]:
%cd /content/
!git clone https://github.com/WellML/detectioncable
%cd detectioncable
!mv cable.names /dn/cable/
!mv cable.data /dn/cable/
!mv yolov3_custom.cfg /dn/cable/
!mv train.txt /dn/cable/
!mv test.txt /dn/cable/
%cd /gdriveyolo/
!cp 02_yolov3_custom_best.weights /dn/cable/
print('OK')

/content
Cloning into 'detectioncable'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 72 (delta 26), reused 8 (delta 2), pack-reused 0
Unpacking objects: 100% (72/72), done.
/content/detectioncable
/content/gdrive/My Drive/yolocable
OK


Darknet wird installiert. Warnungen können ignoriert werden.

In [0]:
%cd /dn/
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warnin

# Vergleich zu Linux:
Nachdem YOLOv3 über das Betriebssystem Linux ein Bild über die Konsole analysierte, wurden dort untereinander die Wahrscheinlichkeiten einer Klasse (bspw. Crimp) aufgelistet. Das Bild im jpg-Format mit den erkannten Klassen wurde im Ordner Darknet gespeichert. Dieses musste jedoch separat nach der Analyse geöffnet werden.

Die folgende Funktion "illustrate" wird genutzt, um nach der Analyse von YOLOv3 sofort das Bild anzeigen zu lassen. [Quelle (3. Test)](https://github.com/robingenz/object-detection-yolov3-google-colab/blob/master/Object_Detection_YOLOv3_Google_Colab.ipynb)

In [0]:
%cd /gdriveyolo/
!cp -r darknet /content/

/content/gdrive/My Drive/yolocable


In [0]:
def illustrate(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

### Test von Darknet und GPU
Nun werden Darknet und die GPU von Colab getestet. Dazu wird die vortrainierte Datei von YOLO **yolov3.weights** verwendet.

In [0]:
#%cd /gdriveyolo
#!/bin/bash
#from pathlib import Path
#my_file = Path("/gdriveyolo/yolov3.weights")
#if not my_file.is_file():
#  !wget https://pjreddie.com/media/files/yolov3.weights
  
#!cp /gdriveyolo/yolov3.weights /dn
#print ('OK')

In [0]:
%cd /dn
#!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg
#illustrate('predictions.jpg')
#!mv /dn/predictions.jpg /gdriveyolo/detected_images

/content/darknet


Ein Bild mit einem Hund, Pferd und einer Person sollte zum Schluss erscheinen.

> Die GPU funktioniert, wenn die Detektionszeit Milli-Sekunden  statt >10 Sekunden benötigt.

# **3) YOLOv3 Custom Cable Detection**
Der Detektor kann nun gestartet werden.

```
!./darknet detect test <path to data> <path to config> <path to weights> <path to image>

-ext_output: Zeige die Position der Output-Box an
-thresh: Threshold (0 bis 1)
```

In [0]:
#< cable/train.txt > result.txt
%cd /dn/
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp1.jpg -dont_show -thresh 0.3 -ext_output < cable/train.txt > result.txt
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp2.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp3.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp4.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp5.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp6.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp7.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp7-2.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp8.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp9.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp10.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp11.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp12.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp13.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp14.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp15.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp16.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
!./darknet detector test cable/cable.data cable/yolov3_custom.cfg cable/02_yolov3_custom_best.weights /gdriveyolo/original_images/bsp17.jpg -dont_show -thresh 0.3 -ext_output 
illustrate('predictions.jpg')
#!mv /dn/predictions.jpg /gdriveyolo/detected_images

# **4) Eigenes Modell trainieren**
Im Folgenden werden die Schritte zum Trainieren der Custom Cable Detection erläutert. Insgesamt werden 6 Dateien generiert bzw. bearbeitet:

>Datei 1) dataset_yolo.zip (Fotos, Label, classes.txt)

>Datei 2) cable.names

>Datei 3) train.txt

>Datei 4) test.txt

>Datei 5) yolov3_custom.cfg

>Datei 6) cable.data

>Datei 7) darknet53.conv.74

Dieser Befehl stoppt den "Run all"-Befehl, indem eine Fehlermeldung angezeigt wird.

In [0]:
stop run-all

SyntaxError: ignored

## Datei 1) - Schritt 1 - Welche Dateien beinhaltet der Datensatz?
Der Datensatz beinhaltet

>mindestens 100 Fotos pro Klasse

>zu den Fotos zugehörige Label als .txt-Datei

>Datei **classes.txt**

## Datei 1) - Schritt 2 - Fotoquelle
Es wurden eigene Fotos mit der Tiefenkamera **[** *Intel Realsense D415* **ändern zu Samsung S10]** aufgenommen.

## Datei 1) - Schritt 3 - Gerenieren von Klassen "**classes.txt**"
Es wurde eine neue Textdatei erstellt mit dem Namen "**classes.txt**". In diesem Projekt lauten die Namen aller Klassen:

> Crimp

> Crimp_Seal

>Isolation

>StrippedIsolation



## Datei 1) - Schritt 4 - Datensatz labeln
Das Labeln des Datensatzes erfolgte durch [labelImg](https://github.com/tzutalin/labelImg). In diesem Programm werden die zu erkennenden Objekte markiert. Dabei erstellt labelImg Textdateien, die im gleichen Bilder-Verzeichnis zu finden sein sollten.

*labelImg wurde auf Ubuntu 18.04 lokal installiert und durchgeführt:*

1) Download und extrahiere zu "Home"
https://github.com/tzutalin/labelImg

2) Eingabe in Ubuntu-Konsole:
> cd ~

> cd labelImg-master

> sudo apt-get install pyqt5-dev-tools

> sudo pip3 install -r requirements/requirements-linux-python3.txt

> make qt5py3

> 3) Die selbsterstellte Datei **classes.txt** in den Ordner mit den Fotos einfügen.

> 4) Eingabe in Ubuntu-Konsole: python3 labelImg.py

labelImg öffnet sich nun.

>5) In "Open Dir" das Verzeichnis auswählen, indem die selbstaufgenomenen Fotos gespeichert sind.

>6) In "Change Save Dir" sicherstellen, dass das selbe Verzeichnis gewählt ist.

>7) Auf "**Pascal VOC**" (.xml) klicken, um auf "**YOLO**" (.txt) Output zu wechseln.

>8) Taste "**W**" wechselt den Zeiger-Cursor zu einem Fadenkreuz-Cursor

>9) Bereich wählen, das durch YOLOv3 später erkannt werden soll.

>10) Eine der vorher definierten Klassen auswählen (siehe Generieren von Klassen).

>11) "**Strg + S**" speichert das gewählte Fenster in für YOLOv3 lesbare Werte.

Beispiel Dateiinhalt Foto1.txt:
**0 0.48 0.50 0.13 0.44**

Klasse = 0 (1. Klasse)
X_Min = 0.48 
X_Max = 0.50
Y_Min = 0.13
Y_Max = 0.44


>12) Taste "**D**" wechselt zum nächsten Foto (Taste "A" zum vorherigen Foto).

## Datei 1) - Schritt 5 - Datensatz auf GDrive hochladen
In diesem Schritt wird der Datensatz bestehend aus Fotos, Label, classes.txt in einem Ordner mit dem Namen "**dataset_yolo.zip**" in das GDrive Verzeichnis "yolocable" manuell hochgeladen. Eine ZIP-Datei verbraucht weniger Speicherplatz und kann schneller hochgeladen werden.

>Gehe zu https://drive.google.com/drive/u/0/my-drive

>Öffne dort den Ordner "**yolocable**"

>Lade **dataset_yolo.zip** in das Verzeichnis "yolocable" hoch.

>Prüfen, ob sich die ZIP-Datei im Pfad aktualisiert hat.

In [0]:
!ls /gdriveyolo

Führe folgenden Code aus, um die ZIP-Datei zu entpacken und zu Darknet zu kopieren. [Quelle](https://medium.com/datadriveninvestor/speed-up-your-image-training-on-google-colab-dc95ea1491cf)



In [0]:
%cd /gdriveyolo/
from pydrive.drive import GoogleDrive
from pathlib import Path
my_file = Path("/gdriveyolo/dataset_yolo.zip")
if my_file.is_file():
  !cp dataset_yolo.zip /dn/custom/
  %cd /dn/custom/
  !unzip /dn/custom/dataset_yolo.zip -d /dn/custom/
  !rm dataset_yolo.zip
  print ('OK')
else:
  print ('ZIP-Datei nicht gefunden.')
!ls /dn/custom

##Datei 2) cable.names

Folgenden Code ausführen, um cable.names zu erstellen.

>Voraussetzung: Die Datei **classes.txt** muss weiterhin im Verzeichnis **dataset_yolo.zip** aus der ZIP-Datei vorhanden sein.

In [0]:
%cd /dn/custom/dataset_yolo/

file = open("classes.txt","r") 
Content = file.read() 
CoList = Content.split("\n")
classes = 0 
for i in CoList: 
    if i: 
        classes += 1

#Anzahl der Klassen
print("Anzahl der Klassen:",classes)

!cp /dn/custom/dataset_yolo/classes.txt /gdriveyolo/cable.names
!cp /gdriveyolo/cable.names /dn/custom

!ls /dn/custom
print('OK')

In [0]:
stop run-all

##Datei 3) train.txt
train.txt ist folgendermaßen aufgebaut:

**custom/dataset_yolo/**Bildname

>Stelle sicher, dass das Bildformat in Zeile 7 stimmt und führe folgenden Code aus. Alle Bilder müssen das selbe Format haben.

In [0]:
import os
%cd /dn/
image_files = []
os.chdir(os.path.join("custom", "dataset_yolo"))

for filename in os.listdir(os.getcwd()):
   if filename.endswith(".jpg"):
       image_files.append("custom/dataset_yolo/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

print('OK')
!ls /dn/custom

##Datei 4) test.txt
In Test.txt sollen 10-20% der Trainingsbilder us train.txt zugeordnet sein.

> Führe folgenden Code aus. Dabei werden automatisch 20% der Zeilen aus train.txt ausgeschnitten und in test.txt eingefügt.


In [0]:
%cd /dn/custom/

file = open("train.txt","r") 
allimg = 0

Content = file.read() 
CoList = Content.split("\n") 

for i in CoList: 
    if i: 
        allimg += 1
print("Anzahl aller Bilder:",allimg)

# Ermittle 10-20% des Datensatzes aus train.txt
testimg = int(round(allimg * 0.15,0))
file.close()
trainimg = allimg-testimg
print("Anzahl Trainbilder:",trainimg)
print("Anzahl Testbilder:",testimg)

# Kopiere 10-20% der Anzahl an Zeilen aus train.txt in test.txt
with open("train.txt",'r') as train:
    data = train.readlines()
with open("train.txt",'w') as train:
    for line in data[testimg:]:
        train.write(line)
with open("test.txt",'w') as test:
    for line in data[:testimg]:
        test.write(line)
test.close() # Close and save the files
train.close()

print('OK')
!ls /dn/custom

## Datei 5) yolov3_custom.cfg



Folgende Änderungen müssen in der Datei **yolov3_custom.txt** vorgenommen werden:

>In Zeile 3 und 4 das **#** vorne hinzufügen (auskommentieren)

>batch=64

>subdivisions=32

>max_batches = [Anzahl der Klassen x 2000; hier: **max_batches = 8000**}

>steps=[max_batches x 80%, x 90%; hier: **steps=6400,7200** ]

**3x in [yolo]:**

> classes=4

> random = 1, außer 1x random = 0, um die Trainingsgeschwindigkeit bei geringfügiger Minderung der Detektionsqualität zu erhöhen.

> num=9

**3x in [convolutional] (jeweils direkt über [yolo]):**

>filters=[ (num/3) * (classes + 5); hier: **filters=27** ]

> Datei speichern (STRG + S)  und schließen.

In [0]:
# IN BEARBEITUNG


#%cd /dn/custom/

#f = open("yolov3_custom.cfg",'r')
#filedata = f.read()
#f.close()

## Anzahl der Klassen anpassen
#newdata = filedata.replace("classes=4","classes=",classes)

## max_batches = [Anzahl der Klassen x 2000; hier: max_batches=3420]
#batches = int(classes * 2000)
#newdata = filedata.replace("max_batches=3420","max_batches=",batches)

## steps=[max_batches x 80%, x 90%; hier: steps=2736,3078 ]
#steps80 = int(batches *0.8)
#steps90 = int(batches *0.9)
#newdata = filedata.replace("steps=2736,3078","steps=",steps80,",",steps90)

## filters = [9/3*(classes+5); hier: filters=27]
#filters = int(9/3*(classes+5))
#newdata = filedata.replace("filters=27","filters=",filters)

#f = open("yolov3_custom.cfg",'w')
#f.write(newdata)
#f.close()

#!sed -i 's/max_batches=3420/max_batches=',classes,'/g' yolov3_custom.cfg

##Datei 5) - Schritt 2 - Kopieren der CFG-Datei zu Darknet
Für Fall 1 und Fall 2 wird ein Uploadfenster geöffnet. Folgender Code kann als Upload nutzen. Dieser Code konvertiert die TXT-Datei zurück in eine CFG-Datei. Danach wird die CFG-Datei zu Darknet kopiert.

>Führe folgenden Code aus.

In [0]:
%cd /gdriveyolo/

def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

from pathlib import Path
my_file2 = Path("/gdriveyolo/yolov3_custom.cfg")
if not my_file2.is_file():
  #Upload yolov3_custom.cfg
  upload()
!cp /dn/cable/yolov3_custom.cfg /dn/custom

!ls /dn/custom
print ('OK')

##Datei 6) cable.data Datei
>Führe den Code aus.

In [0]:
%cd /gdriveyolo/
from pathlib import Path
my_file = Path("/gdriveyolo/cable.data")

if not my_file.is_file():
  data = open('data.txt','w')
  data.write("\r\nclasses = ",classes)
  data.write("\r\ntrain = custom/train.txt")
  data.write("\r\nvalid = custom/test.txt")
  data.write("\r\nnames = custom/cable.names")
  data.write("\r\nbackup = /gdriveyolo/backup/")
  data.close()
  !mv data.txt cable.data
  
!cp /gdriveyolo/cable.data /dn/custom
!ls /dn/custom
print ('OK')

## Datei 7) - Vortrainierte Weights für Convolutional Layers
Die Weights-Datei von der Pjreddie Webseite hilft dabei das Training der benutzerdefinierten Objekterkennung zu beschleunigen.

In [0]:
%cd /gdriveyolo
from pathlib import Path
my_file = Path("/gdriveyolo/darknet53.conv.74")

if not my_file.is_file():
  !wget http://pjreddie.com/media/files/darknet53.conv.74
!cp /gdriveyolo/darknet53.conv.74 /dn/custom/
print('OK')

##**Zusammenfassung für Pfad custom/dn**
Unter folgendem Code sollten nun 7 Elemente eingeblendet werden.

In [0]:
%cd /dn/custom/
!ls /dn/custom/

!cp cable.data /gdriveyolo/backup/
!cp cable.names /gdriveyolo/backup/
!cp yolov3_custom.cfg /gdriveyolo/backup/
!cp train.txt /gdriveyolo/backup/
!cp test.txt /gdriveyolo/backup/

##**Training**

Nun beginnt das Training mit folgendem Code. Dieser Vorgang dauert mehrere Stunden.

In [0]:
%cd /dn/
!./darknet detector train custom/cable.data custom/yolov3_custom.cfg custom/darknet53.conv.74 -dont_show -map 0.02

> Für eine hohe Genauigkeit kann der Trainingsprozess bei unter 0,3 "avg loss" gestoppt werden. Die Datei speichert sich automatisch nach jeden 100 Iterationen (ca. jede 20 Minuten).

>Falls der Prozess ungeplant abbricht, ist der bisherige Fortschritt unter yolocable/backup in GDrive gespeichert. Das Training kann mit folgendem Code fortgeführt werden.

In [0]:
%cd /gdriveyolo/backup/
!cp cable.data /dn/custom/
!cp cable.names /dn/custom/
!cp yolov3_custom.cfg /dn/custom/
!cp train.txt /dn/custom/
!cp test.txt /dn/custom/
%cd /dn/
!./darknet detector train custom/cable.data custom/yolov3_custom.cfg /gdriveyolo/backup/yolov3_custom_last.weights -dont_show -map 0.02

##**Post-Training**


Ändere diese Konfiguration nach dem Training in der Config-Datei.

In [0]:
%cd /gdriveyolo/
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg

Nach dem Training kann der durchschnittliche Verlust (avg loss) gegenüber den Iterationen über das Diagramm  analysiert werden.


In [0]:
!cp chart.png /gdriveyolo/backup/
illustrate('chart.png')

In [0]:
!cp chart_yolov3_custom.png /gdriveyolo/backup/

#**Anwendung**

In [0]:
%cd /dn/
!./darknet detector test /gdriveyolo/cable.data /gdriveyolo/yolov3_custom.cfg /gdriveyolo/backup/yolov3_custom_last.weights /gdriveyolo/original_images/A_01_Color.png -thresh 0.3
illustrate('predictions.jpg')